In [1]:
import sys
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

# Aggiunge la cartella src al path per poter importare i moduli
sys.path.append(os.path.abspath("src"))

# Autoreload per ricaricare i moduli se li modifichi durante lo sviluppo
%load_ext autoreload
%autoreload 2

# Import dei tuoi moduli custom
from data_loader import QuantumDataManager
from quantum_transformer import QuantumTransformer
from curriculum import CurriculumLearning
from utils import ExperimentManager 

In [3]:

TRANSFORMER_CONFIGS = [
    {
        "name": "transformer_small",
        "window_size": 10,
        "forecast_horizon": 1,
        "embed_dim": 32,
        "num_heads": 2,
        "ff_dim": 128,
        "num_layers": 1,
        "dropout": 0.1,
        "learning_rate": 0.001,
        "batch_size": 64
    },
    {
        "name": "transformer_medium",
        "window_size": 15,
        "forecast_horizon": 1,
        "embed_dim": 64,
        "num_heads": 4,
        "ff_dim": 256,
        "num_layers": 2,
        "dropout": 0.2,
        "learning_rate": 0.0005,
        "batch_size": 64
    },
    {
        "name": "transformer_large",
        "window_size": 30,
        "forecast_horizon": 1,
        "embed_dim": 64,
        "num_heads": 4,
        "ff_dim": 256,
        "num_layers": 3,
        "dropout": 0.15,       
        "learning_rate": 0.0003,
        "batch_size": 128
    }
]

In [4]:
def train_with_curriculum(
    model,
    train_ds,
    val_ds,
    optimizer,
    loss_fn,
    epochs,
    tf_epochs=5,
    mm_epochs=8,
    ss_epochs=2,
    ss_max_prob=0.8
):
    curriculum = CurriculumLearning(model, optimizer, loss_fn)

    history = {
        "loss": [],
        "val_loss": [],
        "phase": []
    }

    total_epochs = tf_epochs + mm_epochs + ss_epochs
    assert epochs == total_epochs

    for epoch in range(epochs):
        # --- 1. Determina la fase ---
        if epoch < tf_epochs:
            phase = "teacher_forcing"
            sampling_prob = 0.0
        elif epoch < tf_epochs + mm_epochs:
            phase = "masked_modeling"
            sampling_prob = 0.0
        else:
            phase = "scheduled_sampling"
            ss_progress = (epoch - tf_epochs - mm_epochs) / max(1, ss_epochs - 1)
            sampling_prob = ss_progress * ss_max_prob

        # --- 2. Training Loop ---
        epoch_loss = tf.keras.metrics.Mean()
        
        for x, y in train_ds:
            if phase == "teacher_forcing":
                loss = curriculum.teacher_forcing_step(x, y)
            elif phase == "masked_modeling":
                loss = curriculum.masked_modeling_step(x, y)
            else:
                loss = curriculum.scheduled_sampling_step(x, y, sampling_prob=sampling_prob)
            
            epoch_loss.update_state(loss)

        # --- 3. Validation Loop (NUOVO) ---
        # Calcoliamo la loss sul validation set (in modalità Teacher Forcing standard)
        val_loss_metric = tf.keras.metrics.Mean()
        for x_val, y_val in val_ds:
            preds = model(encoder_input=x_val, decoder_input=y_val, training=False)
            v_loss = loss_fn(y_val, preds)
            val_loss_metric.update_state(v_loss)

        # --- 4. Logging ---
        train_l = epoch_loss.result().numpy()
        val_l = val_loss_metric.result().numpy()
        
        history["loss"].append(train_l)
        history["val_loss"].append(val_l)
        history["phase"].append(phase)

        print(f"Epoch {epoch+1:03d} | {phase[:15]:15s} | Train Loss: {train_l:.5f} | Val Loss: {val_l:.5f}")

    return history

In [ ]:
# Inizializza il manager
exp_manager = ExperimentManager(base_path="data")

def run_curriculum_experiment(config, tf_ep=5, mm_ep=10, ss_ep=10):
    print(f"\n--- Avvio Esperimento: {config['name']} ---")
    
    # 1. Caricamento Dati
    dm = QuantumDataManager(config)
    
    # Assicurati che i dati siano caricati per accedere a X_train
    if not hasattr(dm, 'X_train'):
        dm.load_and_process()

    # Leggi la dim da Numpy invece che dal Dataset ---
    input_dim = dm.X_train.shape[-1] 

    # Ora crea i dataset con cache attiva
    train_ds, val_ds = dm.get_tf_datasets()
    
    print(f"Dati caricati. Input features: {input_dim}")

    # 2. Modello
    model = QuantumTransformer(
        input_dim=input_dim,
        seq_len=config['window_size'],
        d_model=config['embed_dim'],
        num_heads=config['num_heads'],
        d_ff=config['ff_dim'],
        num_layers=config['num_layers'],
        dropout=config['dropout']
    )

    optimizer = tf.keras.optimizers.Adam(learning_rate=config['learning_rate'])
    loss_fn = tf.keras.losses.MeanSquaredError()

    total_epochs = tf_ep + mm_ep + ss_ep

    # 3. Training (con Validation!)
    history = train_with_curriculum(
        model=model,
        train_ds=train_ds,
        val_ds=val_ds,
        optimizer=optimizer,
        loss_fn=loss_fn,
        epochs=total_epochs,
        tf_epochs=tf_ep,
        mm_epochs=mm_ep,
        ss_epochs=ss_ep
    )
    
    # 4. Salvataggio e Plotting (tramite la classe ExperimentManager)
    print("\n--- Salvataggio Risultati ---")
    exp_manager.save_model_artifacts(model, history, config['name'])
    
    print("\n--- Generazione Grafici ---")
    exp_manager.plot_loss_curves(history, config['name'])
    exp_manager.plot_forecast_comparison(model, val_ds, dm, config['name'])
    
    return model, history

# Esecuzione
run_curriculum_experiment(TRANSFORMER_CONFIGS[0])
'''for transformer_config in TRANSFORMER_CONFIGS:
    run_curriculum_experiment(transformer_config)'''


--- Avvio Esperimento: transformer_small ---
Dataset caricato: (400400, 56)
Traiettorie individuate: 400
[transformer_small] Dataset caricato. Train shape: (317120, 10, 55)


2026-01-27 10:39:33.593571: W tensorflow/core/kernels/data/cache_dataset_ops.cc:914] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2026-01-27 10:40:33.738366: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2026-01-27 10:40:52.359199: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 001 | teacher_forcing | Train Loss: 0.01898 | Val Loss: 0.01730


2026-01-27 10:42:11.128412: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 002 | teacher_forcing | Train Loss: 0.01231 | Val Loss: 0.01679
Epoch 003 | teacher_forcing | Train Loss: 0.01220 | Val Loss: 0.01619


KeyboardInterrupt: 

In [ ]:
# GRAFICI
plt.style.use('seaborn-v0_8-whitegrid')

def plot_training_phases_detailed(history, config_name, filename):
    """
    Plotta Loss e MAE con linee verticali allineate perfettamente alle epoche 10 e 20.
    """
    epochs = range(1, len(history['loss']) + 1)

    # Creiamo una figura con 2 grafici affiancati
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))

    # --- GRAFICO 1: LOSS (MSE) ---
    ax1.plot(epochs, history['loss'], label='Train Loss', color='#1f77b4', linewidth=2)
    ax1.plot(epochs, history['val_loss'], label='Validation Loss', color='#ff7f0e', linestyle='--', linewidth=2)

    # Calcolo posizione testo
    y_min, y_max = ax1.get_ylim()
    text_y_pos = y_max - (y_max - y_min) * 0.05

    # --- MODIFICA QUI: Linee verticali su interi esatti ---

    # Fase 1: Standard (Testo centrato su epoca 5)
    ax1.text(5, text_y_pos, 'FASE 1:\nSTANDARD', ha='center', va='top', fontsize=10, fontweight='bold', color='gray', bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.9))

    # Linea su Epoca 10 (Fine Standard / Inizio Masking)
    ax1.axvline(x=10, color='red', linestyle='--', alpha=0.5, linewidth=1.5)

    # Fase 2: Masking (Testo centrato su epoca 15)
    ax1.text(15, text_y_pos, 'FASE 2:\nMASKING', ha='center', va='top', fontsize=10, fontweight='bold', color='gray', bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.9))

    # Linea su Epoca 20 (Fine Masking / Inizio Noise)
    ax1.axvline(x=20, color='red', linestyle='--', alpha=0.5, linewidth=1.5)

    # Fase 3: Noise (Testo centrato su epoca 25)
    ax1.text(25, text_y_pos, 'FASE 3:\nNOISE', ha='center', va='top', fontsize=10, fontweight='bold', color='gray', bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.9))

    ax1.set_title(f'Training Dynamics - {config_name}', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Epoche')
    ax1.set_ylabel('Loss (MSE)')
    ax1.legend(loc='lower left')
    ax1.grid(True, alpha=0.3)

    # Impostiamo i tick dell'asse X per mostrare i numeri chiave
    # Questo forza il grafico a mostrare 1, 10, 20, 30 sull'asse
    ax1.set_xticks([1, 5, 10, 15, 20, 25, 30])

    # --- GRAFICO 2: MAE ---
    ax2.plot(epochs, history['mae'], label='Train MAE', color='#2ca02c', linewidth=2)

    # Linee verticali anche qui (esattamente su 10 e 20)
    ax2.axvline(x=10, color='red', linestyle='--', alpha=0.5)
    ax2.axvline(x=20, color='red', linestyle='--', alpha=0.5)

    ax2.set_title('Mean Absolute Error Evolution', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Epoche')
    ax2.set_ylabel('MAE')
    ax2.set_xticks([1, 5, 10, 15, 20, 25, 30]) # Forza i tick anche qui
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig("../plots/training/" + filename, dpi=300)
    plt.show()

def plot_trajectory_check(model, trajectories_list, window_size, config_name, filename):
    """
    Confronto Predizione vs Realtà su una traiettoria di test.
    """
    # Prendiamo una traiettoria di test a caso (es. indice 0)
    traj_idx = 1
    if len(trajectories_list) > 0:
        real_traj = trajectories_list[traj_idx]

        # Creazione input sequenziale
        X_seq, y_seq = [], []
        for i in range(len(real_traj) - window_size):
            X_seq.append(real_traj[i : i + window_size])
            y_seq.append(real_traj[i + window_size])

        X_seq = np.array(X_seq)
        y_seq = np.array(y_seq)

        # Predizione
        print("Generazione predizioni per il grafico...")
        y_pred = model.predict(X_seq, batch_size=32, verbose=0)

        # Plot
        plt.figure(figsize=(14, 6))
        feature_idx = 0 # Magnetizzazione Z (Feature più importante)

        plt.plot(y_seq[:, feature_idx], label='Realtà (Ground Truth)', color='black', alpha=0.7, linewidth=2)
        plt.plot(y_pred[:, feature_idx], label=f'Predizione ({config_name})', color='#d62728', linestyle='--', linewidth=1.5)

        plt.title(f'Verifica Traiettoria: {config_name} (Window: {window_size})', fontsize=14, fontweight='bold')
        plt.xlabel('Time Steps')
        plt.ylabel('Valore Normalizzato')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.savefig("../plots/predictions/" + filename, dpi=300)
        plt.show()
    else:
        print("Errore: Lista traiettorie vuota.")


In [ ]:
# --- CONFIGURAZIONE PERCORSI ---
MODEL_DIR = '../models_data'
PLOT_DIR = '../plots'
os.makedirs(PLOT_DIR, exist_ok=True)

def run_full_visualization_cycle():
    # Definiamo i gruppi di configurazioni da processare
    config_groups = {
        "rnn": HYPERPARAMETERS_LIST,
        "transf": TRANSFORMER_CONFIGS
    }

    for m_type, configs in config_groups.items():
        for conf in configs:
            name = conf['name']
            safe_name = name.replace(" ", "_").replace("(", "").replace(")", "")
            
            # 1. RECUPERO STORIA (CSV) E GRAFICI LOSS/MAE
            csv_path = f"{MODEL_DIR}/history/{m_type}_{safe_name}.csv"
            if os.path.exists(csv_path):
                print(f"\nGenerando grafici di addestramento per: {name}")
                history_df = pd.read_csv(csv_path)
                # Convertiamo il dataframe in dizionario per le tue funzioni
                history_dict = history_df.to_dict(orient='list')
                
                plot_filename = f"loss_{m_type}_{safe_name}.png"
                plot_training_phases_detailed(history_dict, name, plot_filename)
            else:
                print(f"Storia non trovata per {name} al percorso: {csv_path}")

            # 2. RICARICAMENTO MODELLO E GRAFICI PREDIZIONE
            weights_path = f"{MODEL_DIR}/weights/{m_type}_{safe_name}.weights.h5"
            if os.path.exists(weights_path):
                print(f"Generando grafici di predizione per: {name}")
                
                # Istanziamo l'architettura corretta
                if m_type == "rnn":
                    model = QuantumRNN(hidden_units=conf['units'], output_dim=55, dropout_rate=conf['dropout'])
                else:
                    model = QuantumTransformer(
                        num_layers=conf['num_layers'], embed_dim=conf['embed_dim'],
                        num_heads=conf['num_heads'], ff_dim=conf['ff_dim'],
                        output_dim=55, input_seq_len=conf['window_size'], dropout_rate=conf['dropout']
                    )
                
                # Dummy pass per inizializzare i pesi e caricamento
                dummy_input = tf.random.uniform((1, conf['window_size'], 55))
                _ = model(dummy_input)
                model.load_weights(weights_path)
                
                pred_filename = f"pred_{m_type}_{safe_name}.png"
                # Assicurati che test_traj_norm sia disponibile nel tuo ambiente
                plot_trajectory_check(model, test_traj_norm, conf['window_size'], name, pred_filename)
                
                # Pulizia memoria dopo ogni modello per evitare crash
                tf.keras.backend.clear_session()
            else:
                print(f"Pesi non trovati per {name} al percorso: {weights_path}")

# Avvia il ciclo
run_full_visualization_cycle()